In [4]:
import importlib
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns

from utils import data_utils

RUN_FOLDER_KCM = "../results/debug/kcm/"
RUN_FOLDER_ATB = "../results/debug/atb/"

In [30]:
res = pd.read_csv(f"{RUN_FOLDER_KCM}logs/lightning_logs/version_1/metrics.csv")
res

,FF_train_loss_step,epoch,step,FF_val_loss,FF_train_loss_epoch
0,831.145386,0,49,NaN,NaN
1,NaN,0,55,792.870667,NaN
2,NaN,0,55,NaN,796.160034
3,698.654968,1,99,NaN,NaN
4,NaN,1,111,595.558167,NaN
5,NaN,1,111,NaN,737.077759
6,335.481110,2,149,NaN,NaN
7,NaN,2,167,272.955505,NaN
8,NaN,2,167,NaN,375.503357
9,238.944687,3,199,NaN,NaN


In [26]:
res = pd.read_csv(f"{RUN_FOLDER_KCM}logs/lightning_logs/version_1/metrics.csv")
train_losses = res[['epoch','train_loss_epoch']].dropna()
val_losses = res[['epoch','val_loss']].dropna()
train_losses.merge(val_losses, on="epoch")

KeyError: "['train_loss_epoch'] not in index"

In [25]:
res

,train_loss_step,epoch,step,val_loss,train_loss_epoch
0,831.145386,0,49,NaN,NaN
1,NaN,0,55,792.870667,NaN
2,NaN,0,55,NaN,796.160034
3,698.740479,1,99,NaN,NaN
4,NaN,1,111,598.320190,NaN
5,NaN,1,111,NaN,737.250916
6,394.521851,2,149,NaN,NaN
7,NaN,2,167,366.978699,NaN
8,NaN,2,167,NaN,425.217804
9,368.321686,3,199,NaN,NaN


In [ ]:
# Base results
kcm_model_results = data_utils.load_pkl(RUN_FOLDER_KCM + "kcm/" + "model_results.pkl")
atb_model_results = data_utils.load_pkl(RUN_FOLDER_ATB + "atb/" + "model_results.pkl")
kcm_result_df, kcm_loss_df, kcm_time_df = data_utils.extract_results("Seattle", kcm_model_results)
atb_result_df, atb_loss_df, atb_time_df = data_utils.extract_results("Trondheim", atb_model_results)

In [ ]:
# Include generalization results
kcm_gen_results = data_utils.load_pkl(RUN_FOLDER_KCM + "kcm/" + "model_generalization_results.pkl")
atb_gen_results = data_utils.load_pkl(RUN_FOLDER_ATB + "atb/" + "model_generalization_results.pkl")
kcm_gen_df = data_utils.extract_gen_results("Seattle", kcm_gen_results)
atb_gen_df = data_utils.extract_gen_results("Trondheim", atb_gen_results)

In [ ]:
# Do not include DeepTTE results:
performances_df = pd.concat([kcm_result_df, atb_result_df], axis=0).sort_values(['Model','City','Fold'])
model_curves_df = pd.concat([kcm_loss_df, atb_loss_df], axis=0)
times_df = pd.concat([kcm_time_df, atb_time_df], axis=0)

# # Include DeepTTE results:
# kcm_deeptte_result_df, kcm_deeptte_loss_df, kcm_deeptte_time_df = data_utils.extract_deeptte_results("Seattle", RUN_FOLDER_KCM + "kcm/deeptte_results/result/")
# atb_deeptte_result_df, atb_deeptte_loss_df, atb_deeptte_time_df = data_utils.extract_deeptte_results("Trondheim", RUN_FOLDER_ATB + "atb/deeptte_results/result/")
# performances_df = pd.concat([kcm_result_df, atb_result_df, kcm_deeptte_result_df, atb_deeptte_result_df], axis=0).sort_values(['Model','City','Fold'])
# model_curves_df = pd.concat([kcm_loss_df, atb_loss_df, kcm_deeptte_loss_df, atb_deeptte_loss_df], axis=0)
# times_df = pd.concat([kcm_deeptte_time_df, atb_deeptte_time_df, kcm_time_df, atb_time_df], axis=0)

In [ ]:
# Do not include DeepTTE generalization results
gens_df = pd.concat([kcm_gen_df, atb_gen_df], axis=0)

# # Include DeepTTE generalization results
# kcm_deeptte_gen_results = data_utils.extract_deeptte_gen_results("Seattle", RUN_FOLDER_KCM + "kcm/deeptte_results/generalization/")
# atb_deeptte_gen_results = data_utils.extract_deeptte_gen_results("Trondheim", RUN_FOLDER_ATB + "atb/deeptte_results/generalization/")
# gens_df = pd.concat([kcm_deeptte_gen_results, atb_deeptte_gen_results, kcm_gen_df, atb_gen_df]).sort_values(['Fold','Model'])

In [ ]:
# Baselines for comparing between original evaluation and generalization
kcm_baseline_df = performances_df[performances_df['City']=="Seattle"].copy()
kcm_baseline_df['Loss'] = "Network Baseline"
kcm_baseline_df = kcm_baseline_df[['Model','City','Loss','Fold','MAPE','RMSE','MAE']]
atb_baseline_df = performances_df[performances_df['City']=="Trondheim"].copy()
atb_baseline_df['Loss'] = "Network Baseline"
atb_baseline_df = atb_baseline_df[['Model','City','Loss','Fold','MAPE','RMSE','MAE']]

# Deterioration for comparing change in losses in train network
kcm_deterioration_df = gens_df[gens_df['City']=="Seattle"]
kcm_deterioration_df = kcm_deterioration_df[kcm_deterioration_df['Loss'].isin(['Extract_Train_Losses'])]
atb_deterioration_df = gens_df[gens_df['City']=="Trondheim"]
atb_deterioration_df = atb_deterioration_df[atb_deterioration_df['Loss'].isin(['Extract_Train_Losses'])]

In [ ]:
print(f"All Unique Models: {pd.unique(performances_df['Model'])}")
print(f"All NN Models: {pd.unique(model_curves_df['Model'])}")

### Model Performance

In [ ]:
# plot_data = performances_df[performances_df['City']=="Seattle"]
# fig, axes = plt.subplots(3,1)
# fig.set_figheight(10)
# fig.set_figwidth(10)
# sns.barplot(plot_data, x="MAPE", y="Model", ax=axes[0])
# axes[0].set_xlim([0, 1.0])
# sns.barplot(plot_data, x="RMSE", y="Model", ax=axes[1])
# axes[1].set_xlim([0, 1200])
# sns.barplot(plot_data, x="MAE", y="Model", ax=axes[2])
# axes[2].set_xlim([0, 800])
# fig.suptitle('KCM Model Performance', fontsize=16)
# fig.tight_layout()
# plt.savefig("../plots/model_performances_kcm.png", dpi=1800, bbox_inches='tight')

plot_data = performances_df[performances_df['City']=="Seattle"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(10)
fig.set_figwidth(10)
sns.barplot(plot_data, x="MAE", y="Model", ax=axes)
axes.set_xlim([0, 500])
fig.suptitle('KCM Model Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_performances_kcm.png", dpi=1800, bbox_inches='tight')

In [ ]:
# plot_data = performances_df[performances_df['City']=="Trondheim"]
# fig, axes = plt.subplots(3,1)
# fig.set_figheight(10)
# fig.set_figwidth(10)
# sns.barplot(plot_data, x="MAPE", y="Model", ax=axes[0])
# axes[0].set_xlim([0, 1.0])
# sns.barplot(plot_data, x="RMSE", y="Model", ax=axes[1])
# axes[1].set_xlim([0, 1200])
# sns.barplot(plot_data, x="MAE", y="Model", ax=axes[2])
# axes[2].set_xlim([0, 800])
# fig.suptitle('AtB Model Performance', fontsize=16)
# fig.tight_layout()
# plt.savefig("../plots/model_performances_atb.png", dpi=1800, bbox_inches='tight')

plot_data = performances_df[performances_df['City']=="Trondheim"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(10)
fig.set_figwidth(10)
sns.barplot(plot_data, x="MAE", y="Model", ax=axes)
axes.set_xlim([0, 500])
fig.suptitle('AtB Model Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_performances_atb.png", dpi=1800, bbox_inches='tight')

### Training Time

In [ ]:
plot_data = times_df[times_df['City']=="Seattle"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(12)
sns.barplot(plot_data, x="Time", y="Model", hue="Model", dodge=False)
axes.set_ylabel("Model")
axes.set_xlabel("Training Time (s)")
fig.suptitle('Per-Epoch Model Training Time On KCM', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_train_time_kcm.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = times_df[times_df['City']=="Trondheim"]
fig, axes = plt.subplots(1,1)
fig.set_figheight(6)
fig.set_figwidth(12)
sns.barplot(plot_data, x="Time", y="Model", hue="Model", dodge=False)
axes.set_ylabel("Model")
axes.set_xlabel("Training Time (s)")
fig.suptitle('Per-Epoch Model Training Time On AtB', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_train_time_atb.png", dpi=1800, bbox_inches='tight')

### Training Curves

In [ ]:
model_list = pd.unique(model_curves_df['Model'])
plot_data = model_curves_df[model_curves_df['Model'].isin(model_list[:3])]
g = sns.FacetGrid(data=plot_data, col="City", row="Model", hue="Loss Set", hue_order=["Train","Test"], height=4)
g.map(sns.lineplot, "Epoch", "Loss")
g.add_legend()
plt.savefig(f"../plots/model_curves_1.png", dpi=1800, bbox_inches='tight')

In [ ]:
# model_list = pd.unique(model_curves_df['Model'])
# plot_data = model_curves_df[model_curves_df['Model'].isin(model_list[3:6])]
# g = sns.FacetGrid(data=plot_data, col="City", row="Model", hue="Loss Set", hue_order=["Train","Test"], height=4)
# g.map(sns.lineplot, "Epoch", "Loss")
# g.add_legend()
# plt.savefig(f"../plots/model_curves_2.png", dpi=1800, bbox_inches='tight')

In [ ]:
# model_list = pd.unique(model_curves_df['Model'])
# plot_data = model_curves_df[model_curves_df['Model'].isin(model_list[6:])]
# g = sns.FacetGrid(data=plot_data, col="City", row="Model", hue="Loss Set", hue_order=["Train","Test"], height=4)
# g.map(sns.lineplot, "Epoch", "Loss")
# g.add_legend()
# plt.savefig(f"../plots/model_curves_3.png", dpi=1800, bbox_inches='tight')

### Generalization (Across Networks)

In [ ]:
plot_data = gens_df[gens_df['City']=="Trondheim"]
plot_data = plot_data[plot_data['Loss'].isin(['Test_Losses'])]
plot_data = pd.concat([plot_data, kcm_baseline_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Test_Losses"],["Model Trained on AtB"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(5)
fig.set_figwidth(7)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('KCM Network Generalization Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization_kcm.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = gens_df[gens_df['City']=="Seattle"]
plot_data = plot_data[plot_data['Loss'].isin(['Test_Losses'])]
plot_data = pd.concat([plot_data, atb_baseline_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Test_Losses"],["Model Trained on KCM"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(5)
fig.set_figwidth(7)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('AtB Network Generalization Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_generalization_atb.png", dpi=1800, bbox_inches='tight')

### Generalization (Within Network)

In [ ]:
plot_data = gens_df[gens_df['City']=="Seattle"]
plot_data = plot_data[plot_data['Loss'].isin(['Holdout_Losses'])]
plot_data = pd.concat([plot_data, kcm_baseline_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Holdout_Losses"],["Holdout Routes"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(5)
fig.set_figwidth(7)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('KCM Network Holdout Route Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_holdout_route_kcm.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = gens_df[gens_df['City']=="Trondheim"]
plot_data = plot_data[plot_data['Loss'].isin(['Holdout_Losses'])]
plot_data = pd.concat([plot_data, atb_baseline_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Holdout_Losses"],["Holdout Routes"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(5)
fig.set_figwidth(7)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('AtB Model Holdout Route Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_holdout_route_atb.png", dpi=1800, bbox_inches='tight')

### Fine Tuning/Feature Extraction

In [ ]:
plot_data = gens_df[gens_df['City']=="Trondheim"]
plot_data = plot_data[plot_data['Loss'].isin(['Tune_Test_Losses','Test_Losses'])]
plot_data = pd.concat([plot_data, kcm_baseline_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Tune_Test_Losses","Test_Losses"],["Model Trained on AtB + Tuning", "Model Trained on AtB"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(10)
fig.set_figwidth(10)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('KCM Tuned Network Generalization Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_tuning_kcm.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = gens_df[gens_df['City']=="Seattle"]
plot_data = plot_data[plot_data['Loss'].isin(['Tune_Test_Losses', 'Test_Losses'])]
plot_data = pd.concat([plot_data, atb_baseline_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Tune_Test_Losses","Test_Losses"],["Model Trained on KCM + Tuning", "Model Trained on KCM"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(10)
fig.set_figwidth(10)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('AtB Tuned Network Generalization Performance', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_tuning_atb.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = gens_df[gens_df['City']=="Trondheim"]
plot_data = plot_data[plot_data['Loss'].isin(['Tune_Test_Losses','Test_Losses'])]
plot_data = pd.concat([plot_data, kcm_baseline_df, kcm_deterioration_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Tune_Test_Losses","Test_Losses"],["Model Trained on AtB + Tuning", "Model Trained on AtB"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(10)
fig.set_figwidth(10)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('KCM Network Deterioration', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_deterioration_kcm.png", dpi=1800, bbox_inches='tight')

In [ ]:
plot_data = gens_df[gens_df['City']=="Seattle"]
plot_data = plot_data[plot_data['Loss'].isin(['Tune_Test_Losses','Test_Losses'])]
plot_data = pd.concat([plot_data, atb_baseline_df, atb_deterioration_df])
plot_data = plot_data[plot_data['Model'].isin(['AVG','FF','GRU','TRSF'])]
plot_data['Loss'] = plot_data['Loss'].replace(["Tune_Test_Losses","Test_Losses"],["Model Trained on KCM + Tuning", "Model Trained on KCM"])
fig, axes = plt.subplots(1,1)
fig.set_figheight(10)
fig.set_figwidth(10)
sns.barplot(plot_data, x="MAE", y="Model", hue="Loss", dodge=True)
axes.set_xlim([0, 500])
axes.set_ylabel("Model")
axes.set_xlabel("Model Performance (MAE)")
fig.suptitle('AtB Network Deterioration', fontsize=16)
fig.tight_layout()
plt.savefig("../plots/model_deterioration_atb.png", dpi=1800, bbox_inches='tight')